Now we will compute the potential curves using VQE, H-F and NP but now we will use two different basis sets to compute the electronic integrals: STO-3G and 6-31G.  The latter uses more basis functions to describe molecular orbitals so we expect to get more accurate results than before.

We start in a similar way as before, defining our functions that compute VQE, H-F and NP energies, the only difference is that these functions will now have `basis` as a new input, since we will be looping over the two basis mentioned. 

In [ ]:
# Importing packages neeeded to define the molecule and map the hamiltonian

from qiskit_nature.second_q.drivers import PySCFDriver
from pyscf import scf, gto
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

# First we define a function that takes as input the interatomic distance and returns the mapped hamiltonian, as well as some properties of the problem.

def get_qubit_op(d,basis):

    h2 = MoleculeInfo(["H", "H"], 
                        [(0.0, 0.0, 0.0), (0.0, 0.0, d)], 
                        charge= 0,
                        multiplicity= 1 # 2*spin + 1,
                     )
    
    driver = PySCFDriver.from_molecule(h2 , basis = basis)
    
    problem = driver.run() 
    
    num_particles=problem.num_particles
    
    num_orbitals=int(problem.num_spatial_orbitals)
    
    hamiltonian = problem.hamiltonian
    
    second_q_op = hamiltonian.second_q_op()
    
    mapper=JordanWignerMapper()
    
    converter=QubitConverter(mapper)
    
    qubit_op=converter.convert(second_q_op)

    return qubit_op,problem,converter, num_particles, num_orbitals


# This will compute de SCF-HF energy for the molecule 

def hf_ener(d,basis):
    
    mol = gto.M(
      verbose=0,
      atom=[['H',(0, 0, 0)], 
            ['H',(0, 0, d)]],
      basis=basis,
      charge = 0
    )
    
    mf = scf.RHF(mol).x2c().run()
    hf = mf.kernel()

    return hf


# This next function calculates the exact diagnalization energy

from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
import numpy as np


def np_sol(problem, converter):
    
    solver_ground = NumPyMinimumEigensolverFactory()
    
    calc_gs = GroundStateEigensolver(converter, solver_ground)
    
    np_result_gs = calc_gs.solve(problem)
    
    np_result_gs = np_result_gs.total_energies[0].real
    
    return np_result_gs


# This final one computes the VQE energy.

from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.minimum_eigensolvers import VQE

estimator=Estimator()
slsqp = SLSQP()

def vqe_sol(num_orbitals,num_particles,converter,problem):
    
    init_state = HartreeFock(num_orbitals,num_particles, converter)
    
    uccsd = UCCSD(num_orbitals, num_particles, converter,initial_state=init_state) 
    
    vqe_slsqp = VQE(estimator, uccsd, slsqp)
       
    vqe_solution = vqe_slsqp.compute_minimum_eigenvalue(qubit_op)
       
    vqe_result = problem.interpret(vqe_solution).total_energies[0].real
    
    
    return vqe_result 


Now we define the different lists where we will store our results, as well as the basis to loop over.

In [ ]:
vqe_sto3g = []
vqe_631G = []

np_sto3g = []
np_631G = []

hf_sto3g = []
hf_631G = []


basis = ['6-31G','sto3g']

Now to the main loop, which for each element in the `basis` list, computes the energies using VQE, H-F and NP's exact diagonalization, as well as returninng the computation time and storing the values at each interatomic distance in the previously defined arrays.

In [ ]:
# Start a timer to get the execution time

import timeit

# Main calculation

for b in basis:
    
    start = timeit.default_timer()
    
    for d in distances:
    
        (qubit_op, problem, converter, num_particles, num_orbitals) = get_qubit_op(d, b)
    
        vqe_result = vqe_sol(num_orbitals,num_particles,converter,problem)
    
        np_result = np_sol(problem, converter)

        hf_result = hf_ener(d)
    
        
        if b == 'sto3g':
            
            vqe_sto3g.append(vqe_result)  
    
            np_sto3g.append(np_result)

            hf_sto3g.append(hf_result)
        
        else:
            
            vqe_631G.append(vqe_result)  
    
            np_631G.append(np_result)

            hf_631G.append(hf_result)
    
    
        print(f"Interatomic Distance: {np.round(d, 2)}",
                 f"Basis energy: {np.round(vqe_result,2)}"
                 f"NP energy: {np.round(np_result,2)}")


    print("A set of energies have been calculated")


    stop = timeit.default_timer()

    execution_time = stop - start

    print("Program Executed in "+str(execution_time)) # It returns time in seconds
    

print("All energies have been calculated")

# Save the results in a file

h2_basis = {}
h2_basis['vqe_sto3g'] = vqe_sto3g
h2_basis['np_sto3g'] = np_sto3g
h2_basis['hf_sto3g'] = hf_sto3g
h2_basis['vqe_631G'] = vqe_631G
h2_basis['np_631G'] = np_631G
h2_basis['hf_631G'] = hf_631G

np.savez("h2_basis", **h2_basis)